# Backend

In [1]:
import numpy as np
from itertools import product
from dataclasses import dataclass
import matplotlib.pyplot as plt
import math

In [2]:
significant_figures = 4
_float_tolerance = 5 * (10 ** -(significant_figures +1))

def _do_float_division_with_tolerance(divisor, dividend,):
    if _do_float_eq_with_tolerance(divisor, dividend):
        return int(1)
    
    quotient = np.true_divide(divisor, dividend)
    return quotient

def _do_float_subtraction_with_tolerance(minuend, subtrahend,):
    difference = np.subtract(minuend, subtrahend)

    if _do_float_le_with_tolerance(difference + _float_tolerance, int(difference)) and _do_float_ge_with_tolerance(difference - _float_tolerance, int(difference)):
        return int(difference)

    return difference

vectorized_float_division_with_tolerance = np.vectorize(_do_float_division_with_tolerance)
vectorized_float_subtraction_with_tolerance = np.vectorize(_do_float_subtraction_with_tolerance)

def _do_float_eq_with_tolerance(given, to_compare, sig_figs=significant_figures):
    tolerance = 5 * (10 ** -(significant_figures +1))
    return round(given - tolerance, sig_figs) <= to_compare and round(given + tolerance, sig_figs) >= to_compare

def _do_float_gt_with_tolerance(given, to_compare, sig_figs=significant_figures):
    tolerance = 5 * (10 ** -(significant_figures +1))
    return round(given - tolerance, sig_figs) > to_compare and round(given + tolerance, sig_figs) > to_compare

def _do_float_lt_with_tolerance(given, to_compare, sig_figs=significant_figures):
    tolerance = 5 * (10 ** -(significant_figures +1))
    return round(given - tolerance, sig_figs) < to_compare and round(given + tolerance, sig_figs) < to_compare

def _do_float_ge_with_tolerance(given, to_compare, sig_figs=significant_figures):
    return _do_float_gt_with_tolerance(given, to_compare, sig_figs) or _do_float_eq_with_tolerance(given, to_compare, sig_figs)

def _do_float_le_with_tolerance(given, to_compare, sig_figs=significant_figures):
    return _do_float_lt_with_tolerance(given, to_compare, sig_figs) or _do_float_eq_with_tolerance(given, to_compare, sig_figs)

vectorized_float_eq_with_tolerance = np.vectorize(_do_float_eq_with_tolerance)
vectorized_float_gt_with_tolerance = np.vectorize(_do_float_gt_with_tolerance)
vectorized_float_lt_with_tolerance = np.vectorize(_do_float_lt_with_tolerance)
vectorized_float_ge_with_tolerance = np.vectorize(_do_float_ge_with_tolerance)
vectorized_float_le_with_tolerance = np.vectorize(_do_float_le_with_tolerance)

In [27]:
M=1

def get_curvature_factor(r_coordinate):

    if r_coordinate == 0:
        return np.inf

    schwarzschild_radius = 2*M
    r_coordinate_scaled_by_M = np.multiply(r_coordinate, M)

    quotient = vectorized_float_division_with_tolerance(divisor=schwarzschild_radius, dividend=r_coordinate_scaled_by_M)
    difference = vectorized_float_subtraction_with_tolerance(minuend=np.ones(np.shape(r_coordinate)), subtrahend=quotient)

    if difference > 0:
        return np.sqrt(difference)
    else:
        return -np.sqrt(np.abs(difference))

def get_energy_per_unit_mass_at_shell(r_coordinate)->float:
    return _do_float_subtraction_with_tolerance(1, _do_float_division_with_tolerance(2*M, r_coordinate))

def V_effective(r_coordinate, angular_momentum_per_unit_mass)->float:
    return _do_float_subtraction_with_tolerance(1, _do_float_division_with_tolerance(2*M, r_coordinate))*_do_float_subtraction_with_tolerance(1, -_do_float_division_with_tolerance(angular_momentum_per_unit_mass, r_coordinate)**2)

## The Stone

In [4]:
class Stone:
    def __init__(self, starting_coordinate, *, at_rest, energy_per_unit_mass=0, angular_momentum_per_unit_mass=0,):
        self.position = [starting_coordinate, ]
        if at_rest:
            self.energy_per_unit_mass = get_energy_per_unit_mass_at_shell(self.position[0].polar.r)
            self.angular_momentum_per_unit_mass = angular_momentum_per_unit_mass
        else:
            self.energy_per_unit_mass = energy_per_unit_mass
            self.angular_momentum_per_unit_mass = angular_momentum_per_unit_mass
    
    def get_path_taken(self):
        t_elapsed = [t for t in np.arange(0, len(self.position))]
        path = self.position.copy()
        path.reverse()
        return path, t_elapsed

## Lattice

### Dimensions dataclass

In [5]:
@dataclass(frozen=True)
class Dimensions:
    left: float
    right: float
    top: float
    bottom: float
    resolution: float

@dataclass(frozen=True)
class PolarDimensions:
    inner_r: float
    outer_r: float
    r_resolution: float
    phi_resolution: float
    phi_sig_figs: int

### Coordinate and Coordinate System helper dataclasses

#### Cartesian

In [6]:
@dataclass(frozen=True)
class Cartesian:
    x: int
    y: int

    def __eq__(self, other):
        if isinstance(other, type(self)):
            x_match = vectorized_float_eq_with_tolerance(self.x, other.x)
            y_match = vectorized_float_eq_with_tolerance(self.y, other.y)
            return x_match and y_match
        else:
            return NotImplemented

    def __ge__(self, other):
        if isinstance(other, type(self)):
            x_match = vectorized_float_ge_with_tolerance(self.x, other.x)
            y_match = vectorized_float_ge_with_tolerance(self.y, other.y)
            return x_match and y_match
        else:
            return NotImplemented

    def __le__(self, other):
        if isinstance(other, type(self)):
            x_match = vectorized_float_le_with_tolerance(self.x, other.x)
            y_match = vectorized_float_le_with_tolerance(self.y, other.y)
            return x_match and y_match
        else:
            return NotImplemented

    ## TODO: need to consider case where x == x, y > y
    def __gt__(self, other):
        if isinstance(other, type(self)):
            x_match = vectorized_float_gt_with_tolerance(self.x, other.x)
            y_match = vectorized_float_gt_with_tolerance(self.y, other.y)
            return x_match and y_match
        else:
            return NotImplemented

    def __lt__(self, other):
        if isinstance(other, type(self)):
            x_match = vectorized_float_lt_with_tolerance(self.x, other.x)
            y_match = vectorized_float_lt_with_tolerance(self.y, other.y)
            return x_match and y_match
        else:
            return NotImplemented

#### Polar

In [7]:
@dataclass(frozen=True)
class Polar:
    r: int
    phi: float

    def __eq__(self, other):
        if isinstance(other, type(self)):
            r_match = vectorized_float_eq_with_tolerance(self.r, other.r)
            phi_match = vectorized_float_eq_with_tolerance(self.phi, other.phi)
            return r_match and phi_match
        else:
            return NotImplemented

    def __ge__(self, other):
        if isinstance(other, type(self)):
            r_match = vectorized_float_ge_with_tolerance(self.r, other.r)
            phi_match = vectorized_float_ge_with_tolerance(self.phi, other.phi)
            return r_match and phi_match
        else:
            return NotImplemented

    def __le__(self, other):
        if isinstance(other, type(self)):
            r_match = vectorized_float_le_with_tolerance(self.r, other.r)
            phi_match = vectorized_float_le_with_tolerance(self.phi, other.phi)
            return r_match and phi_match
        else:
            return NotImplemented

    def __gt__(self, other):
        if isinstance(other, type(self)):
            if vectorized_float_eq_with_tolerance(self.r, other.r):
                return vectorized_float_gt_with_tolerance(self.phi, other.phi)

            r_match = vectorized_float_gt_with_tolerance(self.r, other.r)
            phi_match = vectorized_float_gt_with_tolerance(self.phi, other.phi)

            return r_match and phi_match
        else:
            return NotImplemented

    def __lt__(self, other):
        if isinstance(other, type(self)):
            if vectorized_float_eq_with_tolerance(self.r, other.r):
                return vectorized_float_lt_with_tolerance(self.phi, other.phi) 

            r_match = vectorized_float_lt_with_tolerance(self.r, other.r)
            phi_match = vectorized_float_lt_with_tolerance(self.phi, other.phi)
            return r_match and phi_match
        else:
            return NotImplemented

#### Coordinates

In [8]:
class Coordinate:
    def __init__(self, q1: float, q2: float, is_cartesian=True,):
        if is_cartesian:
            self.cartesian = Cartesian(q1, q2)
            self.polar = Coordinate.convert_from_cartesian(self.cartesian)
        else:
            if _do_float_lt_with_tolerance(q2, 0):
                q2 = _do_float_subtraction_with_tolerance(q2, -2*np.pi)
            elif _do_float_gt_with_tolerance(q2, 2*np.pi):
                q2 = _do_float_subtraction_with_tolerance(q2, 2*np.pi)

            self.polar = Polar(q1, q2)
            self.cartesian = Coordinate.convert_from_polar(self.polar)
        self.curvature = get_curvature_factor(self.polar.r)
    
    def convert_from_polar(coordinates_to_convert: Polar):
        x_coordinate = np.multiply(coordinates_to_convert.r, np.cos(coordinates_to_convert.phi))
        y_coordinate = np.multiply(coordinates_to_convert.r, np.sin(coordinates_to_convert.phi))

        return Cartesian(x=x_coordinate, y=y_coordinate)

    def convert_from_cartesian(coordinates_to_convert: Cartesian):
        r_coordinate = np.sqrt(np.power(coordinates_to_convert.x , 2) + np.power(coordinates_to_convert.y , 2))

        if r_coordinate == 0:
            return Polar(r=r_coordinate, phi=0)
        
        phi_coordinate = np.arctan2(coordinates_to_convert.y, coordinates_to_convert.x)

        if _do_float_lt_with_tolerance(phi_coordinate, 0):
            phi_coordinate = _do_float_subtraction_with_tolerance(phi_coordinate, -2*np.pi)

        return Polar(r=r_coordinate, phi=phi_coordinate)
    
    def __eq__(self, other):
        if isinstance(other, type(self)):
            return self.cartesian == other.cartesian and self.polar == other.polar
        else:
            return NotImplemented

    def __gt__(self, other):
        if isinstance(other, type(self)):
            return self.cartesian > other.cartesian and self.polar > other.polar
        else:
            return NotImplemented

    def __ge__(self, other):
        if isinstance(other, type(self)):
           return self.cartesian >= other.cartesian and self.polar >= other.polar
        else:
            return NotImplemented

    def __lt__(self, other):
        if isinstance(other, type(self)):
            return self.cartesian < other.cartesian and self.polar < other.polar
        else:
            return NotImplemented

    def __le__(self, other):
        if isinstance(other, type(self)):
           return self.cartesian <= other.cartesian and self.polar <= other.polar
        else:
            return NotImplemented


### The Latticework Class

#### Creation

In [78]:
class Latticework:

    def __init__(self, /, dimensions: Dimensions=None, polar_dimensions: PolarDimensions=None):
        self.mesh = None
        self.cartesian_dimensions = None
        self.polar_dimensions = None

        if not dimensions is  None:
            self.cartesian_dimensions = dimensions
            self._build_cartesian_grid()
        if not polar_dimensions is None:
            self.polar_dimensions = polar_dimensions
            self._build_polar_grid()

        self._build_dictionary_of_shells()
    
    def _build_cartesian_grid(self):
        self.vertices = { }

        number_of_x_vertices = int((np.abs(self.cartesian_dimensions.left)+np.abs(self.cartesian_dimensions.right))/self.cartesian_dimensions.resolution) +1
        number_of_y_vertices = int(((np.abs(self.cartesian_dimensions.bottom)+np.abs(self.cartesian_dimensions.top)))/self.cartesian_dimensions.resolution) +1

        for x in np.linspace(start=self.cartesian_dimensions.left, stop=self.cartesian_dimensions.right, num=number_of_x_vertices, endpoint=True):
            for y in np.linspace(start=self.cartesian_dimensions.bottom, stop=self.cartesian_dimensions.top, num=number_of_y_vertices, endpoint=True):
                self.vertices.update({(x, y) : Coordinate(q1=x, q2=y)})
        self._build_meshgrid()

    def _build_polar_grid(self):
        self.vertices = { }
        
        number_of_r_vertices = int((np.abs(self.polar_dimensions.inner_r)+np.abs(self.polar_dimensions.outer_r))/self.polar_dimensions.r_resolution) +1
        number_of_phi_vertices = int((2*np.pi)/self.polar_dimensions.phi_resolution)

        r_vertices = np.linspace(start=self.polar_dimensions.inner_r, stop=self.polar_dimensions.outer_r, num=number_of_r_vertices, endpoint=True)
        phi_vertices = np.linspace(start=0, stop=2*np.pi, num=number_of_phi_vertices, endpoint=True)

        for r, phi in product(r_vertices, phi_vertices):
            r = round(r, 1)
            phi = round(phi, self.polar_dimensions.phi_sig_figs)
            if self.vertices.get((r, round(phi, self.polar_dimensions.phi_sig_figs))) is None:
                self.vertices.update({(r, round(phi, self.polar_dimensions.phi_sig_figs)) : Coordinate(q1=r, q2=round(phi, self.polar_dimensions.phi_sig_figs), is_cartesian=False)})
        
        self._build_meshgrid()
    
    def _edge_cost(self, current: Polar, next: Polar, stone_angular_momentum_per_unit_mass: float):
        potential_difference = V_effective(next.r, stone_angular_momentum_per_unit_mass) - V_effective(current.r, stone_angular_momentum_per_unit_mass)
        conservation_of_angular_momentum = stone_angular_momentum_per_unit_mass - (next.phi - current.phi)*(next.r**2)
        return potential_difference - conservation_of_angular_momentum

    def build_graph(self, stone: Stone)->None:

        self.graph = { }

        number_of_r_vertices = int((np.abs(self.polar_dimensions.inner_r)+np.abs(self.polar_dimensions.outer_r))/self.polar_dimensions.r_resolution) +1
        number_of_phi_vertices = int((2*np.pi)/self.polar_dimensions.phi_resolution)

        r_vertices = np.linspace(start=self.polar_dimensions.inner_r, stop=self.polar_dimensions.outer_r, num=number_of_r_vertices, endpoint=True)
        print(r_vertices[1]-r_vertices[0])
        phi_vertices = np.linspace(start=0, stop=2*np.pi, num=number_of_phi_vertices, endpoint=True)
        print(phi_vertices[1]-phi_vertices[0])
        print((2*np.pi)/number_of_phi_vertices)
        for r, phi in product(r_vertices, phi_vertices):
            if _do_float_le_with_tolerance(r, 2*M):
                continue
            r = round(r, 1)
            phi = round(phi, self.polar_dimensions.phi_sig_figs)
            vertex_to_find_neighbors_of = self.vertices.get((r, phi))

            if vertex_to_find_neighbors_of is None:
                print(f'Tried to get the vertex corresponding to ({r=}, {phi=}) to build graph. Latticework.vertices does not have this vertex.')
                break

            vertex_neighbors_with_cost = [ ]
            neighbor_r_vertices = np.linspace(start=r-self.polar_dimensions.r_resolution, stop=r+self.polar_dimensions.r_resolution, num=3)
            neighbor_phi_vertices = np.linspace(start=phi-self.polar_dimensions.phi_resolution, stop=phi+self.polar_dimensions.phi_resolution, num=3)

            for r_i, phi_i in product(neighbor_r_vertices, neighbor_phi_vertices):
                n_i = Coordinate(r_i, phi_i, is_cartesian=False).polar
                r_i, phi_i = round(n_i.r, 1), round(n_i.phi, self.polar_dimensions.phi_sig_figs)
                if (r == 20 and phi == 0.0):
                    print(f'Looking for neighbor at: ({r_i}, {phi_i})')
                neighbor = self.vertices.get((r_i, phi_i))
                if not neighbor is None:
                    if (r == 20 and phi == 0.0):
                        print(f'Adding neighbor ({neighbor.polar.r}, {neighbor.polar.phi})')
                    vertex_neighbors_with_cost.append((neighbor, self._edge_cost(vertex_to_find_neighbors_of.polar, neighbor.polar, stone.angular_momentum_per_unit_mass)))
                else:
                    if (r == 20 and phi == 0.0):
                        print(f'Vertex not found: ({r_i}, {phi_i})')


            self.graph.update({(r, round(phi, self.polar_dimensions.phi_sig_figs)) : vertex_neighbors_with_cost})
    
    def find_path_for_stone(self, stone: Stone):
        stone_position_polar = stone.position[0].polar
        possible_destinations = self.graph[stone_position_polar.r, round(stone_position_polar.phi, self.polar_dimensions.phi_sig_figs) ]
        for neighbor, cost in possible_destinations:
            print(f'{neighbor.polar}, cost: {cost}')
        

    def get_coordinates_of_constant_r(self, constant_r, ascending=True, complete_loop=True) -> list():
        shell_of_constant_r = self.shells["r"][constant_r].copy()
        if not ascending:
            shell_of_constant_r.sort(reverse= not ascending, key=lambda x: x.polar.phi)
        if complete_loop:
            shell_of_constant_r.append(shell_of_constant_r[0])
        return shell_of_constant_r
    
    def get_coordinates_of_constant_phi(self, constant_phi, ascending=True) -> list():
        shell_of_constant_phi = self.shells["phi"][constant_phi].copy()
        if not ascending:
            shell_of_constant_phi.sort(reverse= not ascending, key=lambda x: x.polar.r)

        return shell_of_constant_phi

    def _build_dictionary_of_shells(self):
        self.shells = { 
            "r" : { }, 
            "phi" : { }, 
            }

        sorted_vertices = list(self.vertices.values())
        sorted_vertices.sort(key=lambda x: x.polar.r)

        for coordinate in sorted_vertices:
            if coordinate.polar.r not in self.shells["r"].keys():
                self.shells["r"].update({coordinate.polar.r : [ coordinate, ]})
            else:
                self.shells["r"][coordinate.polar.r].append(coordinate)
        
        sorted_vertices.sort(key=lambda x: x.polar.phi)
        for coordinate in sorted_vertices:
            if coordinate.polar.phi not in self.shells["phi"].keys():
                self.shells["phi"].update({coordinate.polar.phi : [ coordinate, ]})
            else:
                self.shells["phi"][coordinate.polar.phi].append(coordinate)
            
        for r in self.shells["r"].keys():
            self.shells["r"][r].sort(key=lambda x: x.polar.phi)
        
        for phi in self.shells["phi"].keys():
            self.shells["phi"][phi].sort(key=lambda x: x.polar.r)
    
    def get_coordinate_at_edge(self):
        return self.shells["r"][self.dimensions.right][0]

    def _build_meshgrid(self):
        if not self.mesh is None:
            return

        if self.polar_dimensions is None:
            number_of_X_samples = int((np.abs(self.cartesian_dimensions.left)+np.abs(self.cartesian_dimensions.right))/self.cartesian_dimensions.resolution) +1
            number_of_Y_samples = int((np.abs(self.cartesian_dimensions.bottom)+np.abs(self.cartesian_dimensions.top))/self.cartesian_dimensions.resolution) +1

            X = np.linspace(start=self.cartesian_dimensions.left, stop=self.cartesian_dimensions.right, num=number_of_X_samples, endpoint=True)
            Y = np.linspace(start=self.cartesian_dimensions.bottom, stop=self.cartesian_dimensions.top, num=number_of_Y_samples, endpoint=True)
            Z = np.zeros((len(X), len(Y)))

            for i, (x,y) in enumerate(product(X,Y)):
                if _do_float_gt_with_tolerance(self.vertices[x,y].polar.r, 2*M + self.cartesian_dimensions.resolution, sig_figs=1):
                    Z[np.unravel_index(i, (len(X), len(Y)))] = np.true_divide(1, self.vertices[x,y].curvature)

            z_max_index = np.unravel_index(np.argmax(Z), Z.shape)
            Z = np.where(Z == 0, Z[z_max_index]+1, Z)

            X, Y = np.meshgrid(X, Y)

            self.mesh = (X, Y, Z)
            return
        
        if self.cartesian_dimensions is None:
            number_of_r_vertices = int((np.abs(self.polar_dimensions.inner_r)+np.abs(self.polar_dimensions.outer_r))/self.polar_dimensions.r_resolution) +1
            number_of_phi_vertices = int((2*np.pi)/self.polar_dimensions.phi_resolution)

            R = np.linspace(start=self.polar_dimensions.inner_r, stop=self.polar_dimensions.outer_r, num=number_of_r_vertices, endpoint=True)
            Phi = np.linspace(start=0, stop=2*np.pi, num=number_of_phi_vertices, endpoint=True)

            R, Phi = np.meshgrid(R, Phi)
            X, Y = R*np.cos(Phi), R*np.sin(Phi)

            Z = np.zeros(X.shape)
            offset = self.polar_dimensions.r_resolution/2 if self.polar_dimensions.r_resolution >= 1 else self.polar_dimensions.r_resolution

            it = np.nditer([X, Y], flags=['multi_index'])
            for x, y, in it:
                ix, iy = it.multi_index
                r = round(math.sqrt(math.pow(x,2) + math.pow(y,2))*M, int(math.fabs(math.log(self.polar_dimensions.r_resolution))))
                
                if r <= 2.0*M:
                    Z[ix, iy] = 1/get_curvature_factor(2*M+offset/2)
                else:
                    Z[ix, iy] = 1/get_curvature_factor(r)
                
            self.mesh = (X, Y, Z)
            return

        if not self.mesh is None:
            return self.mesh

        #Meshgrid Algorithm from: https://stackoverflow.com/a/32449311

        number_of_X_samples = int((np.abs(self.dimensions.left)+np.abs(self.dimensions.right))/self.dimensions.resolution) +1
        number_of_Y_samples = int((np.abs(self.dimensions.bottom)+np.abs(self.dimensions.top))/self.dimensions.resolution) +1

        X = np.linspace(start=self.dimensions.left, stop=self.dimensions.right, num=number_of_X_samples, endpoint=True)
        Y = np.linspace(start=self.dimensions.bottom, stop=self.dimensions.top, num=number_of_Y_samples, endpoint=True)
        Z = np.zeros((len(X), len(Y)))

        for i, (x,y) in enumerate(product(X,Y)):
            if _do_float_gt_with_tolerance(self.vertices[x,y].polar.r, 2*M + self.dimensions.resolution, sig_figs=1):
                Z[np.unravel_index(i, (len(X), len(Y)))] = np.true_divide(1, self.vertices[x,y].curvature)

        z_max_index = np.unravel_index(np.argmax(Z), Z.shape)
        Z = np.where(Z == 0, Z[z_max_index]+1, Z)

        X, Y = np.meshgrid(X, Y)

        self.mesh = (X, Y, Z)

# Dijkstra's Algorithm

In [79]:
stone = Stone(
        Coordinate(20, 0, is_cartesian=False), 
        at_rest=False,
        energy_per_unit_mass=math.sqrt(V_effective(r_coordinate=20, angular_momentum_per_unit_mass=4)),
        angular_momentum_per_unit_mass=4
    )

lattice_dimensions = PolarDimensions(
        inner_r=0,
        outer_r=21,
        r_resolution=0.1,
        phi_resolution=0.5,
        phi_sig_figs=3
    )

lattice = Latticework(polar_dimensions=lattice_dimensions)



In [76]:
for coord in lattice.get_coordinates_of_constant_r(round(19.9, 1), complete_loop=False):
    print(f'{coord.polar.r}, {coord.polar.phi}')

19.9, 0.0
19.9, 0.571
19.9, 1.142
19.9, 1.714
19.9, 2.285
19.9, 2.856
19.9, 3.427
19.9, 3.998
19.9, 4.57
19.9, 5.141
19.9, 5.712
19.9, 6.283


In [80]:
lattice.build_graph(stone=stone)

0.1
0.5711986642890533
0.5235987755982988
Looking for neighbor at: (19.9, 5.783)
Vertex not found: (19.9, 5.783)
Looking for neighbor at: (19.9, 0.0)
Adding neighbor (19.9, 0.0)
Looking for neighbor at: (19.9, 0.5)
Vertex not found: (19.9, 0.5)
Looking for neighbor at: (20.0, 5.783)
Vertex not found: (20.0, 5.783)
Looking for neighbor at: (20.0, 0.0)
Adding neighbor (20.0, 0.0)
Looking for neighbor at: (20.0, 0.5)
Vertex not found: (20.0, 0.5)
Looking for neighbor at: (20.1, 5.783)
Vertex not found: (20.1, 5.783)
Looking for neighbor at: (20.1, 0.0)
Adding neighbor (20.1, 0.0)
Looking for neighbor at: (20.1, 0.5)
Vertex not found: (20.1, 0.5)


In [64]:

lattice.find_path_for_stone(stone=stone)

Polar(r=19.9, phi=0.0), cost: -4.000160097474824
Polar(r=20.0, phi=0.0), cost: -4.0
Polar(r=20.1, phi=0.0), cost: -3.999840102475174


## 4 Nearest Neighbors

In [ ]:
@dataclass
class Node:
    coordinate: Coordinate
    neighbors: list

class LatticeGraph:
    def __init__(self):
        self.nodes = None